# Set up

Restart run time after running the first cell

In [ ]:
!pip install transformers datasets torch
!pip install sentencepiece
!pip install accelerate -U
!pip install textattack
!pip3 install textattack[tensorflow]
!pip install --upgrade tensorflow

  Using cached tensorflow_text-2.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.2 MB)
INFO: pip is looking at multiple versions of tensorflow-text to determine which version is compatible with other requirements. This could take a while.
  Using cached tensorflow_text-2.14.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.5 MB)
  Using cached tensorflow_text-2.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.5 MB)
  Using cached tensorflow_text-2.12.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.0 MB)
  Using cached tensorflow_text-2.12.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.0 MB)
  Using cached tensorflow_text-2.11.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.8 MB)
  Using cached tensorflow_text-2.10.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.9 MB)
  Using cached tensorflow_text-2.9.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.6 MB)
 

In [ ]:
import torch
from datasets import load_dataset
from transformers import DebertaV2ForSequenceClassification, DebertaV2Tokenizer, Trainer, TrainingArguments
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
if torch.cuda.is_available():
    print(f"GPU available: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not available")

GPU available: Tesla T4


#Before finetuning

Accuracy: 50%

In [ ]:
# Load the tokenizer and model
tokenizer = DebertaV2Tokenizer.from_pretrained('microsoft/deberta-v3-xsmall')
model = DebertaV2ForSequenceClassification.from_pretrained('microsoft/deberta-v3-xsmall')

# Load the RTE dataset
dataset = load_dataset("glue", "qnli")

# Prepare the validation set
validation_set = dataset['validation']

# Function to evaluate the model on the validation set
def evaluate_model(model, tokenizer, validation_set):
    model.eval()  # Set the model to evaluation mode

    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for item in validation_set:
            inputs = tokenizer(item['question'], item['sentence'], return_tensors="pt", truncation=True, padding=True)
            outputs = model(**inputs)
            predictions = torch.argmax(outputs.logits, dim=1)
            correct_predictions += (predictions == item['label']).sum().item()
            total_predictions += 1

    accuracy = correct_predictions / total_predictions
    return accuracy

# Calculate accuracy
accuracy = evaluate_model(model, tokenizer, validation_set)
print(f"Accuracy: {accuracy}")

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/241M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-xsmall and are newly initialized: ['pooler.dense.bias', 'classifier.bias', 'pooler.dense.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generating train split:   0%|          | 0/104743 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5463 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5463 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Accuracy: 0.5053999633900788


#Finetuning

In [ ]:
model_name = "microsoft/deberta-v3-xsmall"
tokenizer = DebertaV2Tokenizer.from_pretrained(model_name)
model = DebertaV2ForSequenceClassification.from_pretrained(model_name, num_labels=2)

dataset = load_dataset("glue", "qnli")
def tokenize_function(examples):
    return tokenizer(examples["question"], examples["sentence"], truncation=True, padding="max_length", max_length=128)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

training_args = TrainingArguments(
    output_dir="./results/qnli",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer
)

trainer.train()


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/241M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-xsmall and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.weight', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Map:   0%|          | 0/5463 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Epoch,Training Loss,Validation Loss
1,0.301500,0.248993
2,0.266800,0.272281


TrainOutput(global_step=26186, training_loss=0.29386595890118294, metrics={'train_runtime': 3277.4799, 'train_samples_per_second': 63.917, 'train_steps_per_second': 7.99, 'total_flos': 3449977244931072.0, 'train_loss': 0.29386595890118294, 'epoch': 2.0})

In [ ]:
save_directory = "/content/drive/My Drive/qnli-deberta-xsmall"

In [ ]:
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('/content/drive/My Drive/qnli-deberta-xsmall/tokenizer_config.json',
 '/content/drive/My Drive/qnli-deberta-xsmall/special_tokens_map.json',
 '/content/drive/My Drive/qnli-deberta-xsmall/spm.model',
 '/content/drive/My Drive/qnli-deberta-xsmall/added_tokens.json')

#Loading model/tokeniser from google drive

In [ ]:
model = DebertaV2ForSequenceClassification.from_pretrained(save_directory)
tokenizer = DebertaV2Tokenizer.from_pretrained(save_directory)

In [ ]:
dataset = load_dataset("glue", "qnli")
validation_dataset = dataset["validation"]

def tokenize_function(examples):
    return tokenizer(examples["question"], examples["sentence"], truncation=True, padding="max_length", max_length=128)

tokenized_validation_dataset = validation_dataset.map(tokenize_function, batched=True)

Generating train split:   0%|          | 0/104743 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5463 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5463 [00:00<?, ? examples/s]

Map:   0%|          | 0/5463 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

#Calculating Accuracy of fine tuned model
Accuracy: 93%

In [ ]:
trainer = Trainer(model=model)

raw_pred, _, _ = trainer.predict(tokenized_validation_dataset)

# Apply softmax and then argmax to get the most likely class
predictions = np.argmax(raw_pred, axis=1)

In [ ]:
true_labels = validation_dataset["label"]

# Analyze the results
for i, (pred, label) in enumerate(zip(predictions, true_labels)):
    if pred != label:
        print(f"Index: {i}, Predicted: {pred}, Actual: {label}")
        print(f"Sentence 1: {validation_dataset[i]['question']}")
        print(f"Sentence 2: {validation_dataset[i]['sentence']}")
        print("-----")

Index: 2, Predicted: 0, Actual: 1
Sentence 1: What is the minimum required if you want to teach in Canada?
Sentence 2: In most provinces a second Bachelor's Degree such as a Bachelor of Education is required to become a qualified teacher.
-----
Index: 6, Predicted: 0, Actual: 1
Sentence 1: What does UMC stand for?
Sentence 2: Founded in 1968 by the union of the Methodist Church (USA) and the Evangelical United Brethren Church, the UMC traces its roots back to the revival movement of John and Charles Wesley in England as well as the Great Awakening in the United States.
-----
Index: 33, Predicted: 0, Actual: 1
Sentence 1: A function problem is an example of what?
Sentence 2: Notable examples include the traveling salesman problem and the integer factorization problem.
-----
Index: 54, Predicted: 0, Actual: 1
Sentence 1: What was the population of the Dutch Republic before this emigration?
Sentence 2: This was a huge influx as the entire population of the Dutch Republic amounted to ca.
-

In [ ]:
accuracy = np.mean(predictions == true_labels)
print(f"Validation Accuracy: {accuracy}")

Validation Accuracy: 0.9287937030935384


# Attack 1 Textfooler

In [ ]:
save_directory = "./finetuned_models/qnli-deberta-xsmall"
model = DebertaV2ForSequenceClassification.from_pretrained(save_directory)
tokenizer = DebertaV2Tokenizer.from_pretrained(save_directory)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import TextFoolerJin2019
from textattack import Attacker, AttackArgs
from textattack.loggers import CSVLogger

In [ ]:
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)
dataset = HuggingFaceDataset("glue", "qnli", split="validation")
attack = TextFoolerJin2019.build(model_wrapper)
attack_args = AttackArgs(
    num_examples=200,
    log_to_csv="log.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    disable_stdout=False
)

In [ ]:
attacker = Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

textattack: Logging to CSV at path log.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|          | 1/200 [00:33<1:52:34, 33.94s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Entailment (99%)]] --> [[Not_entailment (51%)]]

[[[[Question]]]]: What came into force after the new constitution was herald?
[[[[Sentence]]]]: As of that day, the new constitution heralding the Second Republic came into [[force]].

[[[[Question]]]]: What came into force after the new constitution was herald?
[[[[Sentence]]]]: As of that day, the new constitution heralding the Second Republic came into [[strengths]].




[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:   2%|▏         | 3/200 [00:35<38:54, 11.85s/it]

...

--------------------------------------------- Result 199 ---------------------------------------------
[[Entailment (92%)]] --> [[Not_entailment (98%)]]

[[[[Question]]]]: What [[composer]] used Luther's hymns in his works?
[[[[Sentence]]]]: Johann Sebastian Bach included several verses as chorales in his cantatas and based chorale cantatas entirely on them, namely Christ lag in Todes Banden, BWV 4, as early as possibly 1707, in his second annual cycle (1724 to 1725)

[[[[Question]]]]: What [[composers]] used Luther's hymns in his works?
[[[[Sentence]]]]: Johann Sebastian Bach included several verses as chorales in his cantatas and based chorale cantatas entirely on them, namely Christ lag in Todes Banden, BWV 4, as early as possibly 1707, in his second annual cycle (1724 to 1725)




[Succeeded / Failed / Skipped / Total] 180 / 5 / 15 / 200: 100%|██████████| 200/200 [09:23<00:00,  2.82s/it]textattack: Saving checkpoint under "checkpoints/1700958264532.ta.chkpt" at 2023-11-26 00:24:24 after 200 attacks.


--------------------------------------------- Result 200 ---------------------------------------------
[[Not_entailment (84%)]] --> [[Entailment (56%)]]

[[[[Question]]]]: Who studies the expected monetary flow over the life of the project and to monitor the payouts throughout the process?
[[[[Sentence]]]]: The [[presence]] of the mortgage banker is highly likely, even in relatively small [[projects]] since the owner's equity in the property is the most obvious source of funding for a building project.

[[[[Question]]]]: Who studies the expected monetary flow over the life of the project and to monitor the payouts throughout the process?
[[[[Sentence]]]]: The [[attendance]] of the mortgage banker is highly likely, even in relatively small [[programmed]] since the owner's equity in the property is the most obvious source of funding for a building project.







[Succeeded / Failed / Skipped / Total] 180 / 5 / 15 / 200: 100%|██████████| 200/200 [09:23<00:00,  2.82s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 180    |
| Number of failed attacks:     | 5      |
| Number of skipped attacks:    | 15     |
| Original accuracy:            | 92.5%  |
| Accuracy under attack:        | 2.5%   |
| Attack success rate:          | 97.3%  |
| Average perturbed word %:     | 11.19% |
| Average num. words per input: | 38.51  |
| Avg num queries:              | 135.49 |
+-------------------------------+--------+


# Attack 2 - Deepwordbug

In [ ]:
from textattack.attack_recipes import DeepWordBugGao2018

In [ ]:
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)
dataset = HuggingFaceDataset("glue", "qnli", split="validation")
attack = DeepWordBugGao2018.build(model_wrapper)
attack_args = AttackArgs(
    num_examples=200,
    log_to_csv="log.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    disable_stdout=False
)

textattack: Loading datasets dataset glue, subset qnli, split validation.
textattack: Unknown if model of class <class 'transformers.models.deberta_v2.modeling_deberta_v2.DebertaV2ForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
attacker = Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

textattack: Logging to CSV at path log.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterSubstitution(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (3): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    )
  (constraints): 
    (0): LevenshteinEditDistance(
        (max_edit_distance):  30
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|          | 1/200 [00:00<02:23,  1.39it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Entailment (99%)]] --> [[Not_entailment (96%)]]

[[[[Question]]]]: What came into force after the new constitution was herald?
[[[[Sentence]]]]: As of that day, the new constitution heralding the Second Republic came into [[force]].

[[[[Question]]]]: What came into force after the new constitution was herald?
[[[[Sentence]]]]: As of that day, the new constitution heralding the Second Republic came into [[forEce]].




[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:   2%|▏         | 3/200 [00:01<01:40,  1.96it/s]

...

--------------------------------------------- Result 199 ---------------------------------------------
[[Entailment (92%)]] --> [[Not_entailment (91%)]]

[[[[Question]]]]: What composer [[used]] Luther's hymns in his works?
[[[[Sentence]]]]: Johann Sebastian Bach included several verses as chorales in his cantatas and based chorale cantatas entirely on them, namely Christ lag in Todes Banden, BWV 4, as early as possibly 1707, in his second annual cycle (1724 to 1725)

[[[[Question]]]]: What composer [[usd]] Luther's hymns in his works?
[[[[Sentence]]]]: Johann Sebastian Bach included several verses as chorales in his cantatas and based chorale cantatas entirely on them, namely Christ lag in Todes Banden, BWV 4, as early as possibly 1707, in his second annual cycle (1724 to 1725)




[Succeeded / Failed / Skipped / Total] 165 / 20 / 15 / 200: 100%|██████████| 200/200 [03:18<00:00,  1.01it/s]textattack: Saving checkpoint under "checkpoints/1700958782771.ta.chkpt" at 2023-11-26 00:33:02 after 200 attacks.


--------------------------------------------- Result 200 ---------------------------------------------
[[Not_entailment (84%)]] --> [[Entailment (52%)]]

[[[[Question]]]]: Who studies the expected monetary flow over the life of the project and to monitor the payouts throughout the process?
[[[[Sentence]]]]: The presence of the mortgage banker is highly likely, [[even]] in [[relatively]] small projects since the owner's equity in the property is the most obvious source of funding for a building project.

[[[[Question]]]]: Who studies the expected monetary flow over the life of the project and to monitor the payouts throughout the process?
[[[[Sentence]]]]: The presence of the mortgage banker is highly likely, [[ven]] in [[relJtively]] small projects since the owner's equity in the property is the most obvious source of funding for a building project.







[Succeeded / Failed / Skipped / Total] 165 / 20 / 15 / 200: 100%|██████████| 200/200 [03:18<00:00,  1.01it/s]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 165    |
| Number of failed attacks:     | 20     |
| Number of skipped attacks:    | 15     |
| Original accuracy:            | 92.5%  |
| Accuracy under attack:        | 10.0%  |
| Attack success rate:          | 89.19% |
| Average perturbed word %:     | 11.23% |
| Average num. words per input: | 38.51  |
| Avg num queries:              | 51.36  |
+-------------------------------+--------+


# Attack 3 - TextBugger

In [ ]:
import torch
from datasets import load_dataset
from transformers import DebertaV2ForSequenceClassification, DebertaV2Tokenizer, Trainer, TrainingArguments
import numpy as np

In [ ]:
save_directory = "/content/drive/My Drive/qnli-deberta-xsmall/finetuned_models/qnli-deberta-xsmall"
model = DebertaV2ForSequenceClassification.from_pretrained(save_directory)
tokenizer = DebertaV2Tokenizer.from_pretrained(save_directory)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import TextBuggerLi2018
from textattack import Attacker, AttackArgs
from textattack.loggers import CSVLogger

In [ ]:
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)
dataset = HuggingFaceDataset("glue", "qnli", split="validation")
attack = TextBuggerLi2018.build(model_wrapper)
attack_args = AttackArgs(
    num_examples=200,
    log_to_csv="log_textbugger.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    disable_stdout=False
)

textattack: Loading datasets dataset glue, subset qnli, split validation.
textattack: Unknown if model of class <class 'transformers.models.deberta_v2.modeling_deberta_v2.DebertaV2ForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
attacker = Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

textattack: Logging to CSV at path log.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (2): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (3): WordSwapHomoglyphSwap
    (4): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 





  0%|          | 0/200 [01:39<?, ?it/s]



--------------------------------------------- Result 1 ---------------------------------------------
[[Entailment (99%)]] --> [[Not_entailment (97%)]]

[[[[Question]]]]: What came into force after the new constitution was herald?
[[[[Sentence]]]]: As of that day, the new constitution heralding the Second Republic came into [[force]].

[[[[Question]]]]: What came into force after the new constitution was herald?
[[[[Sentence]]]]: As of that day, the new constitution heralding the Second Republic came into [[forϲe]].




--------------------------------------------- Result 2 ---------------------------------------------
[[Not_entailment (99%)]] --> [[Entailment (67%)]]

[[[[Question]]]]: What is the [[first]] major city in the stream of the Rhine?
[[[[Sentence]]]]: The most important tributaries in this area are the Ill below of Strasbourg, the Neckar in Mannheim and the Main across from Mainz.

[[[[Question]]]]: What is the [[frst]] major city in the stream of the Rhine?
[[[[Sentence]]]]: The most important tributaries in this area are the Ill below of Strasbourg, the Neckar in Mannheim and the Main across from Mainz.




...

--------------------------------------------- Result 199 ---------------------------------------------
[[Entailment (92%)]] --> [[Not_entailment (75%)]]

[[[[Question]]]]: What composer [[used]] Luther's hymns in his works?
[[[[Sentence]]]]: Johann Sebastian Bach included several verses as chorales in his cantatas and based chorale cantatas entirely on them, namely Christ lag in Todes Banden, BWV 4, as early as possibly 1707, in his second annual cycle (1724 to 1725)

[[[[Question]]]]: What composer [[usage]] Luther's hymns in his works?
[[[[Sentence]]]]: Johann Sebastian Bach included several verses as chorales in his cantatas and based chorale cantatas entirely on them, namely Christ lag in Todes Banden, BWV 4, as early as possibly 1707, in his second annual cycle (1724 to 1725)






textattack: Saving checkpoint under "checkpoints\1701026033154.ta.chkpt" at 2023-11-26 14:13:53 after 200 attacks.


--------------------------------------------- Result 200 ---------------------------------------------
[[Not_entailment (84%)]] --> [[Entailment (54%)]]

[[[[Question]]]]: Who studies the expected monetary flow over the life of the project and to monitor the payouts throughout the process?
[[[[Sentence]]]]: [[The]] presence of the mortgage banker is highly likely, even in relatively small projects since the owner's equity in the property is the most obvious source of funding for a building project.

[[[[Question]]]]: Who studies the expected monetary flow over the life of the project and to monitor the payouts throughout the process?
[[[[Sentence]]]]: [[Per]] presence of the mortgage banker is highly likely, even in relatively small projects since the owner's equity in the property is the most obvious source of funding for a building project.







[Succeeded / Failed / Skipped / Total] 155 / 30 / 15 / 200: 100%|██████████| 200/200 [1:33:21<00:00, 28.01s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 155    |
| Number of failed attacks:     | 30     |
| Number of skipped attacks:    | 15     |
| Original accuracy:            | 92.5%  |
| Accuracy under attack:        | 15.0%  |
| Attack success rate:          | 83.78% |
| Average perturbed word %:     | 22.33% |
| Average num. words per input: | 38.51  |
| Avg num queries:              | 77.95  |
+-------------------------------+--------+


# Attack 4 - PWWS

In [ ]:
import torch
from datasets import load_dataset
from transformers import DebertaV2ForSequenceClassification, DebertaV2Tokenizer, Trainer, TrainingArguments
import numpy as np

save_directory = "/content/drive/My Drive/finetuned_models/qnli-deberta-xsmall"
model = DebertaV2ForSequenceClassification.from_pretrained(save_directory)
tokenizer = DebertaV2Tokenizer.from_pretrained(save_directory)

from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import PWWSRen2019
from textattack import Attacker, AttackArgs
from textattack.loggers import CSVLogger

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)
dataset = HuggingFaceDataset("glue", "qnli", split="validation")
attack = PWWSRen2019.build(model_wrapper)
attack_args = AttackArgs(
    num_examples=200,
    log_to_csv="log_PWWS.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    disable_stdout=False
)

Generating train split:   0%|          | 0/104743 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5463 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5463 [00:00<?, ? examples/s]

textattack: Loading datasets dataset glue, subset qnli, split validation.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
textattack: Unknown if model of class <class 'transformers.models.deberta_v2.modeling_deberta_v2.DebertaV2ForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
attacker = Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

textattack: Logging to CSV at path log_PWWS.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|          | 1/200 [00:06<21:15,  6.41s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Entailment (99%)]] --> [[Not_entailment (98%)]]

[[[[Question]]]]: What came into force after the new constitution was herald?
[[[[Sentence]]]]: As of that day, the new constitution heralding the Second Republic came into [[force]].

[[[[Question]]]]: What came into force after the new constitution was herald?
[[[[Sentence]]]]: As of that day, the new constitution heralding the Second Republic came into [[pressure]].




[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:   2%|▏         | 3/200 [00:07<08:41,  2.65s/it]

...

--------------------------------------------- Result 199 ---------------------------------------------
[[Entailment (92%)]] --> [[Not_entailment (88%)]]

[[[[Question]]]]: What composer [[used]] Luther's hymns in his works?
[[[[Sentence]]]]: Johann Sebastian Bach included several verses as chorales in his cantatas and based chorale cantatas entirely on them, namely Christ lag in Todes Banden, BWV 4, as early as possibly 1707, in his second annual cycle (1724 to 1725)

[[[[Question]]]]: What composer [[victimized]] Luther's hymns in his works?
[[[[Sentence]]]]: Johann Sebastian Bach included several verses as chorales in his cantatas and based chorale cantatas entirely on them, namely Christ lag in Todes Banden, BWV 4, as early as possibly 1707, in his second annual cycle (1724 to 1725)




[Succeeded / Failed / Skipped / Total] 159 / 26 / 15 / 200: 100%|██████████| 200/200 [13:40<00:00,  4.10s/it]textattack: Saving checkpoint under "checkpoints/1701043927744.ta.chkpt" at 2023-11-27 00:12:07 after 200 attacks.


--------------------------------------------- Result 200 ---------------------------------------------
[[Not_entailment (84%)]] --> [[Entailment (57%)]]

[[[[Question]]]]: Who studies the expected monetary flow over the life of the project and to monitor the payouts throughout the process?
[[[[Sentence]]]]: The presence of the mortgage banker is highly likely, even in relatively small [[projects]] since the owner's equity in the property is the most obvious source of funding for a building project.

[[[[Question]]]]: Who studies the expected monetary flow over the life of the project and to monitor the payouts throughout the process?
[[[[Sentence]]]]: The presence of the mortgage banker is highly likely, even in relatively small [[contrive]] since the owner's equity in the property is the most obvious source of funding for a building project.







[Succeeded / Failed / Skipped / Total] 159 / 26 / 15 / 200: 100%|██████████| 200/200 [13:40<00:00,  4.10s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 159    |
| Number of failed attacks:     | 26     |
| Number of skipped attacks:    | 15     |
| Original accuracy:            | 92.5%  |
| Accuracy under attack:        | 13.0%  |
| Attack success rate:          | 85.95% |
| Average perturbed word %:     | 9.48%  |
| Average num. words per input: | 38.51  |
| Avg num queries:              | 237.89 |
+-------------------------------+--------+


In [ ]:
import torch
from datasets import load_dataset
from transformers import DebertaV2ForSequenceClassification, DebertaV2Tokenizer, Trainer, TrainingArguments
import numpy as np

save_directory = "/content/drive/My Drive/finetuned_models/qnli-deberta-xsmall"
model = DebertaV2ForSequenceClassification.from_pretrained(save_directory)
tokenizer = DebertaV2Tokenizer.from_pretrained(save_directory)

from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import PSOZang2020
from textattack import Attacker, AttackArgs
from textattack.loggers import CSVLogger

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)
dataset = HuggingFaceDataset("glue", "qnli", split="validation")
attack = PSOZang2020.build(model_wrapper)
attack_args = AttackArgs(
    num_examples=200,
    log_to_csv="qnli_log_PSO.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    disable_stdout=False
)

Generating train split:   0%|          | 0/104743 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5463 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5463 [00:00<?, ? examples/s]

textattack: Loading datasets dataset glue, subset qnli, split validation.
textattack: Downloading https://textattack.s3.amazonaws.com/transformations/hownet/word_candidates_sense.pkl.
100%|██████████| 8.39M/8.39M [00:00<00:00, 13.4MB/s]
textattack: Copying /root/.cache/textattack/tmpnj0o7z9b.zip to /root/.cache/textattack/transformations/hownet/word_candidates_sense.pkl.
textattack: Successfully saved transformations/hownet/word_candidates_sense.pkl to cache.
textattack: Unknown if model of class <class 'transformers.models.deberta_v2.modeling_deberta_v2.DebertaV2ForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
attacker = Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

textattack: Logging to CSV at path qnli_log_PSO.csv


Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  60
    (max_iters):  20
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapHowNet(
    (max_candidates):  -1
  )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
    (2): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (columns_to_ignore):  {'premise'}
      )
  (is_black_box):  True
) 



  0%|          | 0/200 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/72.9M [00:00<?, ?B/s]

2023-11-29 20:16:06,242 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, VERB, PUNCT, ADP, DET, PROPN, PRON, ADJ, ADV, CCONJ, PART, NUM, AUX, INTJ, SYM, X, <START>, <STOP>


...

--------------------------------------------- Result 199 ---------------------------------------------
[[Entailment (92%)]] --> [[Not_entailment (66%)]]

[[[[Question]]]]: What composer used Luther's hymns in his works?
[[[[Sentence]]]]: Johann Sebastian Bach included several [[verses]] as chorales in his cantatas and based chorale cantatas entirely on them, namely Christ lag in Todes Banden, BWV 4, as early as possibly 1707, in his second annual cycle (1724 to 1725)

[[[[Question]]]]: What composer used Luther's hymns in his works?
[[[[Sentence]]]]: Johann Sebastian Bach included several [[epics]] as chorales in his cantatas and based chorale cantatas entirely on them, namely Christ lag in Todes Banden, BWV 4, as early as possibly 1707, in his second annual cycle (1724 to 1725)




[Succeeded / Failed / Skipped / Total] 79 / 106 / 15 / 200: 100%|██████████| 200/200 [1:19:56<00:00, 23.98s/it]textattack: Saving checkpoint under "checkpoints/1701293756805.ta.chkpt" at 2023-11-29 21:35:56 after 200 attacks.


--------------------------------------------- Result 200 ---------------------------------------------
[[Not_entailment (84%)]] --> [[Entailment (56%)]]

[[[[Question]]]]: Who studies the expected monetary flow over the life of the project and to monitor the payouts throughout the [[process]]?
[[[[Sentence]]]]: The presence of the mortgage banker is highly likely, even in relatively small projects since the owner's equity in the property is the most obvious [[source]] of funding for a building project.

[[[[Question]]]]: Who studies the expected monetary flow over the life of the project and to monitor the payouts throughout the [[course]]?
[[[[Sentence]]]]: The presence of the mortgage banker is highly likely, even in relatively small projects since the owner's equity in the property is the most obvious [[fountain]] of funding for a building project.







[Succeeded / Failed / Skipped / Total] 79 / 106 / 15 / 200: 100%|██████████| 200/200 [1:19:56<00:00, 23.98s/it]


+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 79      |
| Number of failed attacks:     | 106     |
| Number of skipped attacks:    | 15      |
| Original accuracy:            | 92.5%   |
| Accuracy under attack:        | 53.0%   |
| Attack success rate:          | 42.7%   |
| Average perturbed word %:     | 9.12%   |
| Average num. words per input: | 38.51   |
| Avg num queries:              | 6013.48 |
+-------------------------------+---------+
